In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_5/models/B5R2b5_AN_1FC_fold5_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold5.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4601, 25)
(4601, 25)
Normal:  (3005, 25)
Abnormal:  (1596, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4601 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-1.68628275e-01, -1.05422877e-01,  2.70550787e-01, ...,
         3.30982206e-04,  1.54463686e-02, -1.20941907e-01],
       [-1.02615289e-01, -1.00563377e-01,  3.75555158e-01, ...,
         9.04116482e-02,  1.34229824e-01, -1.39623046e-01],
       [-1.53301254e-01, -9.35952961e-02,  1.21402003e-01, ...,
        -9.70871672e-02, -7.04906881e-02, -8.96713585e-02],
       ...,
       [ 1.02357507e+00,  5.14638245e-01, -1.58617586e-01, ...,
         5.25158882e-01, -6.61723688e-03,  8.83743346e-01],
       [ 4.27108049e-01,  6.57781303e-01, -1.40116200e-01, ...,
         1.03076696e+00, -2.06721261e-01,  1.04461551e+00],
       [ 1.25274703e-01,  7.12132156e-01, -1.76603734e-01, ...,
         8.14773560e-01, -1.48888543e-01,  6.62952483e-01]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.168628,-0.105423,0.270551,0.028317,-0.211911,-0.085060,0.083504,-0.138048,-0.174466,-0.140155,...,-0.094257,-0.078666,0.026005,-0.093261,-0.081267,0.055103,-0.059580,0.000331,0.015446,-0.120942
1,-0.102615,-0.100563,0.375555,0.179506,-0.202863,0.053044,0.299665,-0.135393,-0.143777,-0.088539,...,-0.055413,0.129114,0.078015,-0.020453,0.036432,-0.034561,-0.061908,0.090412,0.134230,-0.139623
2,-0.153301,-0.093595,0.121402,0.058033,-0.216119,0.045422,0.204417,-0.220251,-0.196129,-0.199197,...,-0.083553,0.067345,0.102598,-0.113001,-0.013766,-0.099897,-0.146006,-0.097087,-0.070491,-0.089671
3,-0.162324,-0.105784,0.110982,0.008772,-0.219105,-0.086781,0.041028,-0.170106,-0.201664,-0.195727,...,-0.069680,-0.059620,-0.068177,-0.108006,-0.076934,0.050365,-0.079158,-0.095547,-0.085685,-0.149904
4,-0.159389,-0.119691,0.421336,0.046177,-0.182873,0.150922,0.562665,-0.169549,-0.172541,-0.113588,...,-0.081009,0.094381,0.008186,-0.104003,-0.114709,0.067873,-0.071732,-0.036064,-0.025462,-0.153869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,1.068731,1.025799,-0.134295,-0.105713,0.823317,-0.169376,-0.111787,0.293477,0.946880,0.489999,...,0.242093,-0.167866,-0.121939,1.110421,0.234903,-0.167450,-0.071906,0.967422,-0.166078,0.733675
4597,0.851066,0.095680,-0.087423,0.125713,0.443799,-0.123616,-0.185888,0.461973,0.183871,0.697665,...,0.444526,-0.129800,-0.046779,0.471481,-0.086043,-0.186200,-0.130488,0.075347,-0.154827,0.562096
4598,1.023575,0.514638,-0.158618,-0.076200,0.441495,0.059489,-0.066986,0.588306,0.672361,0.323859,...,0.660033,-0.164252,-0.076890,1.004111,-0.072821,-0.138486,0.104547,0.525159,-0.006617,0.883743
4599,0.427108,0.657781,-0.140116,-0.094766,0.798813,-0.186997,-0.183781,0.848097,0.690396,0.860322,...,0.544576,-0.192363,-0.073398,0.563963,-0.074656,-0.069324,-0.161411,1.030767,-0.206721,1.044616


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.168628,-0.105423,0.270551,0.028317,-0.211911,-0.085060,0.083504,-0.138048,-0.174466,-0.140155,...,-0.081267,0.055103,-0.059580,0.000331,0.015446,-0.120942,Abnormal,P31,FP-B,AB01
1,-0.102615,-0.100563,0.375555,0.179506,-0.202863,0.053044,0.299665,-0.135393,-0.143777,-0.088539,...,0.036432,-0.034561,-0.061908,0.090412,0.134230,-0.139623,Abnormal,P1,FP-A,AB01
2,-0.153301,-0.093595,0.121402,0.058033,-0.216119,0.045422,0.204417,-0.220251,-0.196129,-0.199197,...,-0.013766,-0.099897,-0.146006,-0.097087,-0.070491,-0.089671,Abnormal,P2,FP-A,AB01
3,-0.162324,-0.105784,0.110982,0.008772,-0.219105,-0.086781,0.041028,-0.170106,-0.201664,-0.195727,...,-0.076934,0.050365,-0.079158,-0.095547,-0.085685,-0.149904,Abnormal,P42,FP-B,AB01
4,-0.159389,-0.119691,0.421336,0.046177,-0.182873,0.150922,0.562665,-0.169549,-0.172541,-0.113588,...,-0.114709,0.067873,-0.071732,-0.036064,-0.025462,-0.153869,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,1.068731,1.025799,-0.134295,-0.105713,0.823317,-0.169376,-0.111787,0.293477,0.946880,0.489999,...,0.234903,-0.167450,-0.071906,0.967422,-0.166078,0.733675,Normal,P32,FP-B,Normal
4597,0.851066,0.095680,-0.087423,0.125713,0.443799,-0.123616,-0.185888,0.461973,0.183871,0.697665,...,-0.086043,-0.186200,-0.130488,0.075347,-0.154827,0.562096,Normal,P42,FP-B,Normal
4598,1.023575,0.514638,-0.158618,-0.076200,0.441495,0.059489,-0.066986,0.588306,0.672361,0.323859,...,-0.072821,-0.138486,0.104547,0.525159,-0.006617,0.883743,Normal,P52,FP-C,Normal
4599,0.427108,0.657781,-0.140116,-0.094766,0.798813,-0.186997,-0.183781,0.848097,0.690396,0.860322,...,-0.074656,-0.069324,-0.161411,1.030767,-0.206721,1.044616,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold5_1.csv') #เปลี่ยนชื่อไฟล์